In [1]:
from jesse import helpers, research

warmup_candles, trading_candles = research.get_candles(
    "Binance Perpetual Futures",
    "BTC-USDT",
    "3m",
    helpers.date_to_timestamp("2021-01-01"),
    helpers.date_to_timestamp("2025-01-31"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)

In [4]:
from offline.labeling.labeling import (
    TripleBarrierLabeler,
    expand_labels,
    return_of_label,
)

for i in [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01]:
    num_minutes = 16
    labeler = TripleBarrierLabeler(
        trading_candles, num_minutes=num_minutes, verbose=False
    )
    target_ret = i
    side_labels = labeler.side_labels(target_ret=target_ret)
    meta_labels = labeler.meta_labels(side_labels, target_ret=target_ret)
    candle_df = expand_labels(meta_labels, trading_candles, fill=0)
    ret = return_of_label(candle_df)
    label_count = candle_df["side"].value_counts().sort_index().to_numpy()
    print(f"{target_ret = } -> {ret = }, {label_count = }")

target_ret = 0.001 -> ret = 34.69478391925583, label_count = array([ 18584, 664157,  18059])
target_ret = 0.002 -> ret = 34.13375791282092, label_count = array([ 17315, 666834,  16651])
target_ret = 0.003 -> ret = 31.96094493095482, label_count = array([ 14676, 672229,  13895])
target_ret = 0.004 -> ret = 28.29259259084941, label_count = array([ 11476, 678369,  10955])
target_ret = 0.005 -> ret = 23.983245708121622, label_count = array([  8725, 683586,   8489])
target_ret = 0.006 -> ret = 20.096281838951278, label_count = array([  6656, 687627,   6517])
target_ret = 0.007 -> ret = 16.57265425088558, label_count = array([  5090, 690839,   4871])
target_ret = 0.008 -> ret = 13.554155769403627, label_count = array([  3894, 693140,   3766])
target_ret = 0.009 -> ret = 11.17917069259449, label_count = array([  2969, 694954,   2877])
target_ret = 0.01 -> ret = 9.513092482053338, label_count = array([  2424, 696080,   2296])


In [3]:
from offline.labeling.labeling import (
    TripleBarrierLabeler,
    expand_labels,
    return_of_label,
)

num_minutes = 16
labeler = TripleBarrierLabeler(trading_candles, num_minutes=num_minutes, verbose=True)
target_ret = 0.002
side_labels = labeler.side_labels(target_ret=target_ret)
meta_labels = labeler.meta_labels(side_labels, target_ret=target_ret)
candle_df = expand_labels(meta_labels, trading_candles, fill=0)
ret = return_of_label(candle_df)
label_count = candle_df["side"].value_counts().sort_index().to_numpy()
print(f"{target_ret = } -> {ret = }, {label_count = }")

2025-02-12 00:14:51.638968 100.0% _apply_pt_sl_on_t1 done after 0.03 minutes. Remaining 0.0 minutes..
2025-02-12 00:15:55.871380 100.0% _apply_pt_sl_on_t1 done after 0.03 minutes. Remaining 0.0 minutes..


target_ret = 0.002 -> ret = 30.042119901925254, label_count = array([ 17671, 680987,  17022])


In [4]:
trading_candles.shape

(715680, 6)

In [5]:
print(candle_df.shape)
candle_df.head()

(715680, 5)


,close,ret,trgt,bin,side
2021-01-01 00:00:00+00:00,29016.23,NaN,NaN,0.0,0.0
2021-01-01 00:03:00+00:00,28965.02,NaN,NaN,0.0,0.0
2021-01-01 00:06:00+00:00,28928.06,NaN,NaN,0.0,0.0
2021-01-01 00:09:00+00:00,28851.82,NaN,NaN,0.0,0.0
2021-01-01 00:12:00+00:00,28786.75,NaN,NaN,0.0,0.0


In [6]:
candle_df["trgt"].isna().sum()

181593

In [7]:
candle_df.to_csv("data/label_d002.csv.zst")